Include all the packages needed for this exercise as reported below

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import math 
import uproot
import pandas

Here two classes where one defines the caracteristics of a sample (name and sample identification), the other defines which setup to be fitted (center of the ellipse, radius or both) 

In [ ]:
class Samples:
    Signal_NonRes = -125
    TT = 1
        
class FitSetup:
    x0y0 = 1
    ab = 2
    x0y0ab = x0y0 + ab
    
class Regions:
    OS_Isolated = 1
    OS_AntiIsolated = 2
    SS_Isolated = 3
    SS_AntiIsolated = 4

In the following parts open the files, called "anaTuples". Create a DataFrame for the tree.

Specify for the two samples we want to analyze, Signal SM and TT and create a map for the names.

In [ ]:
#path = "/gpfs/ddn/cms/user/cmsdas/2019/hh_bbtautau/anaTuples/" #in Pisa
#path = "/eos/user/m/mgrippo/CMSDAS_2019_hh_bbtautau/anaTuples/" #on Swan

channel = #choose the channel with which you want to work

branches = [ 'sample_id', 'region_id', 'm_sv', 'm_bb', 'weight', 'm_tt_vis']

#open the file with uproot, create a pandas DataFrame (as in Significance Estimation exercise), but only for 
#selected branches

#skimmed DataFrame to be in the signal region (OS_Isolated)
df = df[((df.sample_id == Samples.Signal_NonRes) | (df.sample_id == Samples.TT)) \ 
        & (df.region_id == Regions.OS_Isolated)]


samples = [Samples.Signal_NonRes,Samples.TT]
sample_names = { Samples.Signal_NonRes : "Signal", Samples.TT : "TT"} #only for plotting visualization

Make the plot for both Signal and TT of mass of Higgs into bb candidate and of Higgs into tau pair candidate, in order to view the different behaviour of the signal and of the background. What can you notice?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,10))

n = 0
bins_x = np.linspace(0, 400, 20)
bins_y = np.linspace(0, 400, 20)
for sample in samples:
    #create a hist2d with pyplot, specifying the sample_id for each variable, 
    #set axis labels and title of the histogram
    
    n += 1

plt.show()

Write the code for the minimization of the ellipse cut around mbb and mtautau. It has to be first calculated with 2 paramenters (the center of the ellipse or the radius of the ellipse). Then consider also the possiblity to have a fit of 4 parameters

In [ ]:
#This class is inherited from the base class for the target function in the ROOT minimization framework 
class EllipseMinimization( ROOT.TPyMultiGenFunction ):
    def __init__( self, samples, x0, y0, a, b, fit_setup):
        #store parameters in the instance of the class
        ROOT.TPyMultiGenFunction.__init__( self, self )

    def NDim(self):
        """Return number of dimensions"""

    def DoEval(self, args):
        n = 0
        """Evaluate the loss function in the point provided by args. Args is an array of NDim dimensions"""
        #define the elements and the args according to which setup should be run  (center or radius)
        
        #calculate the number of signal and bkg events
        
        #calculate the significance as in the previous exercise to obtain the minimum   
    
        return # -significance
        
    def CalcNumEvents(self, sample_id, x0, y0, a, b):
        #define the number of events inside the ellipse 
        #it is important to apply weights!

Here the function for minimization, where you have to pass the tolerance from outside. Here reported the default value for tolerance. You should change it for the fit of each parameter.

In [ ]:
def FindParams(samples, x0, y0, a, b, fit_setup):
    minimizer = ROOT.Math.Factory.CreateMinimizer("Minuit", "Combined")
    minimizer.SetTolerance(0.1) #The default tolerance value is 0.1, and the minimization will stop 
                                #when the estimated vertical distance to the minimum (EDM) is less than
                                #0.001∗tolerance 
    minimizer.SetPrintLevel(2)
    minimizer.SetStrategy(2)
    ellipse = #create EllipseMinimization

    minimizer.SetFunction(ellipse)

    # Set the variables to be minimized!
    c_factor = #choose reasonable values for center variable limits
    r_factor = #choose reasonable values for radius variable limits
    n = 0
    if fit_setup & FitSetup.x0y0 != 0:
        minimizer.SetLimitedVariable(0, "x0", x0, 1, x0 * (1 - c_factor), x0 * (1 + c_factor))
        minimizer.SetLimitedVariable(1, "y0", y0, 1, y0 * (1 - c_factor), y0 * (1 + c_factor))
        n = 2
    if fit_setup & FitSetup.ab != 0:
        minimizer.SetLimitedVariable(n, "a", a, 0.1, max(2, a * (1 - r_factor)), a * (1 + r_factor))
        minimizer.SetLimitedVariable(n + 1, "b", b, 0.1, max(2, b * (1 - r_factor)), b * (1 + r_factor))
        
    minimizer.Minimize()
    if(!minimizer.Minimize()) :
        raise RuntimeError("Minimization is not converged.")
    #result = [x for ix in minimizer.X()]
    result = [minimizer.X()[i] for i in range(ellipse.NDim())]
    cov_matrix = np.zeros((ellipse.NDim(), ellipse.NDim()))
    for k in range(ellipse.NDim()):
        for l in range(ellipse.NDim()):
            cov_matrix[k, l] = minimizer.CovMatrix(k, l)
    return tuple(result), cov_matrix

Now you have to choose a starting point for the minimization. 
For the center we suggest to use the most probable value, which can be calculated in the similar way as in Significance Estimation exercise

In [ ]:
sgn_id = Samples.Signal_NonRes
#calculate mpv
print(x0, y0)

For the radius as the initial values we suggest to use an estimate based on the interquantile distance (distance between 25th and 75th percentiles), similar to the previous exercise.

In [ ]:
#calculate the radius
print(a, b)

Calculate new center parameters starting from the initial values calculated for radius and center

In [ ]:
#use function FindParams with correct setup, print as a table new values and error from cov_matrix
print(new_x0, new_y0)

Calculate new radius parameters starting from the starting point for radius and the calculated one for centers

In [ ]:
#use FindParams with correct setup, print as a table new values and error from cov_matrix
print(new_a, new_b)

Calculate the final parameters fitting the 4 parameters, using as initial values the center and radius values obtained by previus fit in 2 dimensions

In [ ]:
#use FindParams in 4 dimension, print as a table new values and error from cov_matrix
print(new_x0, new_y0, new_a, new_b)
print(final_x0, final_y0, final_a, final_b)

Try now to use instead of m_sv the visible invariant mass for H->tautau candidate